In [ ]:
from mec_env_var import *
from helper import *
import tensorflow.compat.v1 as tf
import tflearn
import ipdb as pdb
import time

for k in range(5):
    
    tf.reset_default_graph()
    
    print('---------' + str(k) + '------------')
    
    MAX_EPISODE = 2000
    MAX_EPISODE_LEN = 200

    NUM_T = 1
    NUM_R = 4
    SIGMA2 = 1e-9

    t_factor = 0.2
    noise_sigma = 0.12

    config = {'state_dim':3, 'action_dim':2};
    train_config = {'minibatch_size':64, 'actor_lr':0.0001, 'tau':0.001, 
                    'critic_lr':0.001, 'gamma':0.99, 'buffer_size':250000, 
                    'random_seed':int(time.perf_counter()*1000%1000), 'noise_sigma':noise_sigma, 'sigma2':SIGMA2}
    
    IS_TRAIN = False
#     rate = 3.0
    res_path = 't_M_02_nB/'
    model_path = 'model_M_02_nB/my_train_model_'+str(k)+'-2000'
    meta_path = model_path+'.meta'
    init_path = ''
    init_seqCnt = 40
    
    user_config = [{'id':'1', 'model':'AR', 'num_r':NUM_R, 'rate':1.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000},
                   {'id':'2', 'model':'AR', 'num_r':NUM_R, 'rate':2.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000},
                   {'id':'3', 'model':'AR', 'num_r':NUM_R, 'rate':3.0, 'dis':100, 'action_bound':2, 
                    'data_buf_size':100, 't_factor':t_factor, 'penalty':1000}]
    
    if not os.path.exists(res_path):
        os.mkdir(res_path) 
    
    print(user_config)
    # 0. initialize the session object
    sess = tf.Session() 

    # gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.2)
    # sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 

    # 1. include all user in the system according to the user_config
    user_list = [];
    for info in user_config:
        info.update(config)
        info['model_path'] = model_path + '_' + info['id']
        info['meta_path'] = info['model_path']+'.meta'
        info['init_path'] = init_path
        info['init_seqCnt'] = init_seqCnt
        user_list.append(MecTermRL(sess, info, train_config))
        print('Initialization OK!----> user ' + info['id'])

    # 2. create the simulation env
    env = MecSvrEnv(user_list, NUM_R, SIGMA2, MAX_EPISODE_LEN)

    sess.run(tf.global_variables_initializer())
    
    tflearn.config.is_training(is_training=IS_TRAIN, session=sess)

    env.init_target_network()

    res_r = []
    res_p = []
    res_b = []
    res_o = []
    res_d = []
    # 3. start to explore for each episode
    for i in range(MAX_EPISODE):

        cur_init_ds_ep = env.reset()

        cur_r_ep = np.zeros(len(user_list))
        cur_p_ep = np.zeros(len(user_list))
        cur_op_ep = np.zeros(len(user_list))
        cur_ts_ep = np.zeros(len(user_list))
        cur_ps_ep = np.zeros(len(user_list))
        cur_rs_ep = np.zeros(len(user_list))
        cur_ds_ep = np.zeros(len(user_list))
        cur_ch_ep = np.zeros(len(user_list))
        cur_of_ep = np.zeros(len(user_list))

        for j in range(MAX_EPISODE_LEN):

            # first try to transmit from current state
            [cur_r, done, cur_p, cur_op, temp, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of] = env.step_transmit()

            cur_r_ep += cur_r
            cur_p_ep += cur_p
            cur_op_ep += cur_op
            cur_ts_ep += cur_ts
            cur_ps_ep += cur_ps
            cur_rs_ep += cur_rs
            cur_ds_ep += cur_ds
            cur_ch_ep += cur_ch
            cur_of_ep += cur_of


            if done:
                res_r.append(cur_r_ep/MAX_EPISODE_LEN)
                res_p.append(cur_p_ep/MAX_EPISODE_LEN)
                res_b.append(cur_ds_ep/MAX_EPISODE_LEN)
                res_o.append(cur_of_ep/MAX_EPISODE_LEN)
                res_d.append(cur_ds)
                print('%d:r:%s,p:%s,op:%s,tr:%s,pr:%s,rev:%s,dbuf:%s,ch:%s,ibuf:%s,rbuf:%s' % (i, cur_r_ep/MAX_EPISODE_LEN, cur_p_ep/MAX_EPISODE_LEN, cur_op_ep/MAX_EPISODE_LEN, cur_ts_ep/MAX_EPISODE_LEN, cur_ps_ep/MAX_EPISODE_LEN, cur_rs_ep/MAX_EPISODE_LEN, cur_ds_ep/MAX_EPISODE_LEN, cur_ch_ep/MAX_EPISODE_LEN, cur_init_ds_ep, cur_ds))

    name = res_path+'test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
    np.savez(name, res_r, res_p, res_b, res_o, res_d)
    
    tflearn.config.is_training(is_training=False, session=sess)
    #Create a saver object which will save all the variables
    saver = tf.train.Saver() 
    saver.save(sess, model_path)

    sess.close()



Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)

---------0------------
[{'id': '1', 'model': 'AR', 'num_r': 4, 'rate': 1.0, 'dis': 100, 'action_bound': 2, 'data_buf_size': 100, 't_factor': 0.2, 'penalty': 1000}, {'id': '2', 'model': 'AR', 'num_r': 4, 'rate': 2.0, 'dis': 100, 'action_bound': 2, 'data_buf_size': 100, 't_factor': 0.2, 'penalty': 1000}, {'id': '3', 'model': 'AR', 'num_r': 4, 'rate': 3.0, 'dis': 100, 'action_bound': 2, 'data_buf_size': 100, 't_factor': 0.2, 'penalty': 1000}]
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Initialization OK!----> user 1
Initialization OK!----

16:r:[ -80.85124315  -31.55388919 -182.84121069],p:[0.01181851 1.51090136 0.15283497],op:[0.         0.57432284 0.        ],tr:[0.02143529 1.08052085 0.        ],pr:[0.         1.8609925  1.05927912],rev:[1.09  1.835 3.045],dbuf:[101.03450766  34.22930099 228.16942594],ch:[3.18564538 3.64183217 3.70536127],ibuf:[0.0, 36.5, 37.0],rbuf:[213.76370781   1.61135589 434.14417561]
17:r:[-104.13053964   -9.33639716 -162.9238687 ],p:[0.04250244 2.11120273 1.30179402],op:[0.         1.76056515 0.        ],tr:[0.         2.31536408 0.94679181],pr:[0.42196377 2.5125154  1.04004169],rev:[1.035 1.935 2.87 ],dbuf:[130.05691844   1.99107675 200.40035083],ch:[3.51404926 4.59676484 4.00961125],ibuf:[45.5, 6.0, 2.5],rbuf:[168.10724669   2.         179.13330106]
18:r:[-15.41069581  -9.95594765 -11.16999934],p:[0.22690988 2.27458104 2.71477915],op:[0.03491713 1.72509938 0.45615246],tr:[0.09411554 2.16980363 1.71521023],pr:[1.21200697 2.51970758 2.28607248],rev:[1.015 1.965 3.245],dbuf:[18.60880225  2.44573

39:r:[ -7.28206403  -9.54053308 -10.54165089],p:[0.68022587 2.25400171 2.69375784],op:[1.30787034 1.74437366 0.90098248],tr:[0.         2.30886565 1.53116684],pr:[2.51470748 2.5198421  2.51947745],rev:[1.01  1.84  2.945],dbuf:[4.13233949 1.92972792 4.1902128 ],ch:[4.83625687 4.85582661 2.77360243],ibuf:[45.0, 0.5, 11.5],rbuf:[0. 1. 4.]
40:r:[-4.90512143 -9.80522637 -9.9821113 ],p:[0.60189803 2.45954911 2.45218322],op:[0.85271892 1.50368308 1.17711376],tr:[0.         1.83668453 1.89830804],pr:[2.24565    2.50515504 2.50166617],rev:[1.    2.165 2.915],dbuf:[2.49485941 2.34845249 3.40439668],ch:[2.71158694 3.26268556 4.08410605],ibuf:[30.0, 0.0, 4.0],rbuf:[1.         2.98847323 5.        ]
41:r:[ -2.65029676 -10.28525326 -11.82750233],p:[0.38177879 2.26866341 2.97177982],op:[0.18700882 1.56553349 0.96971252],tr:[0.         2.12349552 1.8834967 ],pr:[1.54514651 2.49259322 2.51326386],rev:[0.915 2.105 3.08 ],dbuf:[1.89090193 3.27107433 4.93064706],ch:[4.38461906 4.01843156 3.70090295],ibuf:

62:r:[ -2.26196273  -9.80062008 -10.45282043],p:[0.38267006 2.24068897 2.58603653],op:[0.00352725 1.60705963 1.07288654],tr:[0.         2.04390549 1.99111854],pr:[1.1036765  2.5198421  2.48532727],rev:[1.015 2.1   3.12 ],dbuf:[1.86196015 2.63140362 3.91871788],ch:[3.3089719  4.28202829 4.06516481],ibuf:[9.0, 23.5, 13.5],rbuf:[0.35820427 4.         7.46698923]
63:r:[ -2.1304319  -11.11310405 -10.53858781],p:[0.41186111 2.314665   2.63087659],op:[0.04121948 1.32677439 1.08303995],tr:[0.         2.03971294 1.92308596],pr:[1.37521182 2.43169537 2.5198421 ],rev:[1.095 2.225 3.04 ],dbuf:[1.53033841 4.78778161 3.8884434 ],ch:[3.52333747 3.94933874 4.06270532],ibuf:[9.0, 45.0, 17.5],rbuf:[1. 4. 1.]
64:r:[ -4.09113322  -8.8093213  -13.69055618],p:[0.51749491 2.01299739 2.78202211],op:[0.00164011 1.47337633 0.92790315],tr:[0.03671109 2.18181161 1.79951574],pr:[1.195112   2.42284339 2.5198421 ],rev:[1.    1.97  2.995],dbuf:[3.81607898 2.29571731 7.83838208],ch:[4.35360779 4.52189651 3.24442118],i

85:r:[ -3.51141189  -9.29856689 -10.842767  ],p:[0.6784702  2.24112114 2.71036713],op:[0.36579024 1.55581013 1.25030573],tr:[0.8250861  2.1166363  2.10530645],pr:[1.62334756 2.45146175 2.519806  ],rev:[0.955 2.01  2.925],dbuf:[1.77861377 2.13088045 3.6517766 ],ch:[3.83033946 4.05760668 3.39079211],ibuf:[32.5, 9.0, 14.5],rbuf:[0. 2. 3.]
86:r:[ -3.21521569  -8.97249446 -11.12025635],p:[0.75657829 2.09634722 2.85043234],op:[0.23058387 1.33604413 0.93865764],tr:[1.14489403 1.74993408 1.76255559],pr:[1.39094726 2.42410484 2.5198421 ],rev:[0.995 2.055 3.045],dbuf:[1.55111421 2.6346397  4.42759549],ch:[4.72452152 3.23041522 3.10474409],ibuf:[21.0, 18.5, 21.5],rbuf:[1.         1.         5.50562231]
87:r:[ -2.38978074  -8.92293301 -10.95582763],p:[0.6225497  2.087181   2.69405761],op:[0.14784266 1.43277228 1.00167178],tr:[0.98679782 1.81318927 1.80932985],pr:[1.13703639 2.45630721 2.5198023 ],rev:[1.01  2.155 2.91 ],dbuf:[1.06124503 2.35378307 4.45546106],ch:[4.48175512 3.52410351 3.87415323],

108:r:[ -1.86977033 -10.10783662 -11.02012928],p:[0.35312344 2.24152826 2.86951416],op:[0.00287685 1.66227401 0.48782029],tr:[0.7007184  2.24060226 1.70658838],pr:[0.32261383 2.5198421  2.31347288],rev:[0.875 1.985 3.315],dbuf:[1.44721221 2.87529009 5.38182549],ch:[5.77905432 4.10843678 3.16229756],ibuf:[4.5, 36.5, 10.0],rbuf:[ 1.48067215  0.         12.34449706]
109:r:[ -3.37380584  -9.4896914  -10.11514561],p:[0.6873252  2.15639426 2.49639439],op:[5.11371715e-04 1.58218199e+00 8.57173400e-01],tr:[0.76034254 1.92454091 2.00539327],pr:[0.60506591 2.5198421  2.32226445],rev:[1.125 2.085 3.01 ],dbuf:[2.49766586 2.51567362 4.26001255],ch:[4.42878278 4.28396102 4.04257589],ibuf:[35.0, 25.5, 14.0],rbuf:[1.95518473 1.         3.        ]
110:r:[ -3.03660271 -10.34314765 -10.83820459],p:[0.48293612 2.2315515  2.66117721],op:[5.86389972e-05 1.42435709e+00 1.00601716e+00],tr:[0.20997642 1.67793045 2.09252244],pr:[0.8789458  2.5198421  2.40844938],rev:[1.015 2.075 2.835],dbuf:[2.58826649 3.78916

131:r:[ -3.08286976 -11.09152037 -11.78218644],p:[0.51132486 2.3495482  3.03917795],op:[9.44452320e-04 1.32915330e+00 8.43564522e-01],tr:[0.36506228 1.99418306 1.79256782],pr:[0.86384868 2.45064844 2.48546251],rev:[1.055 2.14  3.105],dbuf:[2.57291391 4.66764671 5.02087687],ch:[4.15827749 3.89275983 3.40708899],ibuf:[35.0, 44.5, 18.0],rbuf:[3.12046676 0.         3.        ]
132:r:[ -2.36045236  -9.25302101 -13.33883778],p:[0.39002523 2.0840213  3.02752655],op:[0.05356133 1.63645248 0.73920457],tr:[0.15133569 2.25423165 1.82970303],pr:[1.20997652 2.49708098 2.43627091],rev:[0.91  2.135 3.135],dbuf:[1.84159906 2.2650918  7.25671943],ch:[3.40420252 5.5123477  3.54059342],ibuf:[32.0, 5.0, 42.5],rbuf:[1.28202691 5.14805486 4.19287842]
133:r:[ -2.3490131   -9.16910563 -10.51352806],p:[0.40909936 2.09131587 2.60833449],op:[0.11315137 1.62813964 1.26419227],tr:[0.12566346 1.88488423 2.25392659],pr:[1.38183209 2.51719726 2.4860404 ],rev:[0.885 1.93  2.9  ],dbuf:[1.63063955 2.16274325 3.46059318]

154:r:[ -2.75184319  -9.56191761 -10.6141383 ],p:[0.46921727 2.31211517 2.77718635],op:[0.03045606 1.56611194 1.01987721],tr:[0.31924148 2.34344505 2.2615339 ],pr:[1.10898123 2.46705268 2.43053027],rev:[1.08 2.13 3.22],dbuf:[2.19062068 2.25682923 3.77501396],ch:[3.97721918 4.80751412 4.24895187],ibuf:[31.5, 2.5, 6.0],rbuf:[1.08532917 1.         4.        ]
155:r:[ -3.12255501  -9.99736595 -10.8581868 ],p:[0.53885611 2.38529426 2.68518887],op:[0.03993069 1.395149   1.16189912],tr:[0.24349938 1.93982206 2.28570594],pr:[1.31986193 2.43664161 2.45372475],rev:[1.225 2.16  2.955],dbuf:[2.45622677 3.0455993  3.95501352],ch:[4.55796163 4.07046184 5.17838554],ibuf:[34.0, 33.5, 24.5],rbuf:[1.1930178 7.        2.       ]
156:r:[ -1.86310046 -10.31341235 -12.45205992],p:[0.31814647 2.33852262 2.88713759],op:[0.06585333 1.4090176  1.05787709],tr:[0.10973261 2.14256983 1.99947895],pr:[1.25289998 2.41361146 2.49626562],rev:[0.995 2.025 2.855],dbuf:[1.36887607 3.52291488 5.7025382 ],ch:[3.974668   3.9

177:r:[ -3.39710048  -9.68487164 -11.64968752],p:[0.63392481 2.29091787 2.75391133],op:[0.01129649 1.54677264 1.04713361],tr:[0.66361646 2.03326325 2.14576787],pr:[0.67923464 2.51825414 2.43274217],rev:[1.025 2.24  3.   ],dbuf:[2.63332234 2.51186327 5.05949705],ch:[4.30210661 4.35662595 4.1772124 ],ibuf:[46.5, 14.5, 19.5],rbuf:[0.26791328 3.         6.95736939]
178:r:[ -3.06873643 -10.78130918 -10.9902963 ],p:[0.66315918 2.44943751 2.73944307],op:[0.00696575 1.5362916  0.92264876],tr:[1.24245953 2.11533148 2.17772342],pr:[0.35259184 2.51839817 2.36912755],rev:[1.035 2.135 2.97 ],dbuf:[2.16060822 3.5123137  4.58264079],ch:[6.30767916 3.53537648 3.94547854],ibuf:[37.0, 41.0, 34.5],rbuf:[1.         2.         3.61355257]
179:r:[ -2.62668151 -10.01581292 -12.6780828 ],p:[0.65065945 2.35659451 2.92951493],op:[0.02743675 1.64340549 0.97633652],tr:[0.82466326 2.4181181  2.31910989],pr:[0.53934678 2.5198421  2.4796126 ],rev:[1.035 1.985 3.2  ],dbuf:[1.58811139 2.51976615 6.08297487],ch:[4.2104

In [ ]:
l = range(2,3)
print(l)

In [ ]:
name = 't_10/test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
a,b,c,d = plot_everything(res_rec, win=10)
np.savez(name, a,b,c,d)

In [ ]:
name = 't_10/test_1000_' + time.strftime("%b_%d_%Y_%H_%M_%S", time.localtime(time.time()))
a,b,c,d = plot_everything(res_rec, win=10)
np.savez(name, a,b,c,d)

In [ ]:
cur_init_ds_ep = env.reset()
    
cur_r_ep = np.zeros(len(user_list))
cur_p_ep = np.zeros(len(user_list))
cur_n_ep = np.zeros(len(user_list))
cur_ts_ep = np.zeros(len(user_list))
cur_rs_ep = np.zeros(len(user_list))
cur_ps_ep = np.zeros(len(user_list))
cur_ds_ep = np.zeros(len(user_list))
cur_ch_ep = np.zeros(len(user_list))
    
for j in range(1000):
    # first try to transmit from current state
    [cur_r, done, cur_p, cur_n, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of] = env.step_transmit()
        
    cur_r_ep += cur_r
    cur_p_ep += cur_p
    cur_ts_ep += cur_ts
    cur_ps_ep += cur_ps
    cur_rs_ep += cur_rs
    cur_ds_ep += cur_ds
    cur_ch_ep += cur_ch
        
    print('%d:r:%f,p:%s,n:%s,tr:%s,pr:%s,rev:%s,dbuf:%s,ch:%s,ibuf:%s' % (j, cur_r, cur_p, cur_n, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_init_ds_ep))
        
print('r:%f,p:%s,n:%s,tr:%s,rev:%s,dbuf:%s,ch:%s,ibuf:%s' % (cur_r_ep/MAX_EPISODE_LEN, cur_p_ep/MAX_EPISODE_LEN, cur_n_ep/MAX_EPISODE_LEN, cur_ts_ep/MAX_EPISODE_LEN, cur_rs_ep/MAX_EPISODE_LEN, cur_ds_ep/MAX_EPISODE_LEN, cur_ch_ep/MAX_EPISODE_LEN, cur_init_ds_ep))

In [ ]:
graph = tf.get_default_graph()
inputs = graph.get_tensor_by_name("input_1"+":0")
scaled_out = graph.get_tensor_by_name("output_1"+":0")
print sess.run(scaled_out, feed_dict={inputs: [[7.22254768,2.3384,2.7784888e-05]]})

In [ ]:
res_rec = []
res_init_buf = []
# 3. start to explore for each episode
for i in range(MAX_EPISODE):
    
    cur_init_ds_ep = env.reset(3)
        
    res_init_buf.append(cur_init_ds_ep)
    
    cur_r_ep = np.zeros(len(user_list))
    cur_p_ep = np.zeros(len(user_list))
    cur_ts_ep = np.zeros(len(user_list))
    cur_ps_ep = np.zeros(len(user_list))
    cur_rs_ep = np.zeros(len(user_list))
    cur_ds_ep = np.zeros(len(user_list))
    cur_ch_ep = np.zeros(len(user_list))
    cur_of_ep = np.zeros(len(user_list))
    
    res_rec_tmp = []
    for j in range(MAX_EPISODE_LEN):
        
        # first try to transmit from current state
        [cur_r, done, cur_p, temp, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of] = env.step_transmit(False)
        
        res_rec_tmp.append([cur_r, done, cur_p, temp, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of])
        cur_r_ep += cur_r
        cur_p_ep += cur_p
        cur_ts_ep += cur_ts
        cur_ps_ep += cur_ps
        cur_rs_ep += cur_rs
        cur_ds_ep += cur_ds
        cur_ch_ep += cur_ch
        cur_of_ep += cur_of
        
        if done:
            res_rec.append(res_rec_tmp)
            print('%d:r:%.4f,p:%.4f,tr:%.4f,pr:%.4f,rev:%.4f,dbuf:%.4f,ch:%.8f,ibuf:%d' % (i, cur_r_ep/MAX_EPISODE_LEN, cur_p_ep/MAX_EPISODE_LEN, cur_ts_ep/MAX_EPISODE_LEN, cur_ps_ep/MAX_EPISODE_LEN, cur_rs_ep/MAX_EPISODE_LEN, cur_ds_ep/MAX_EPISODE_LEN, cur_ch_ep/MAX_EPISODE_LEN, cur_init_ds_ep[0]))
                
name = 'test_1000_' + str(time.asctime( time.localtime(time.time()) )) + '.npy'
np.save(name, [res_rec, res_init_buf])

In [ ]:
from mec_env_var import *
from helper import *
import tensorflow.compat.v1 as tf
import ipdb as pdb
import time

MAX_EPISODE = 50
MAX_EPISODE_LEN = 1000

NUM_T = 1
NUM_R = 1
SIGMA2 = 0.3e-9

config = {'state_dim':3, 'action_dim':2};
train_config = {'minibatch_size':64, 'actor_lr':0.0001, 'tau':0.001, 
                 'critic_lr':0.001, 'gamma':0.99, 'buffer_size':250000, 
                 'random_seed':1234}
user_config = [{'id':'1', 'rate':3, 'dis':100, 'action_bound':1, 
                'data_buf_size':100, 't_factor':0.9, 'penalty':1000}];

print user_config
# 0. initialize the session object
sess = tf.Session() 
saver = tf.train.import_meta_graph('model/my_test_model.meta')
sess.run(tf.global_variables_initializer())

saver.restore(sess, tf.train.latest_checkpoint('model/'))

graph = tf.get_default_graph()
    
# 1. include all user in the system according to the user_config
user_list = [];
for info in user_config:
    info.update(config)
    user_list.append(MecTermLD(sess, info, train_config))
    print 'test'

# 2. create the simulation env
env = MecSvrEnv(user_list, NUM_R, SIGMA2, MAX_EPISODE_LEN)
env.init_target_network()

res_rec = []
res_init_buf = []
# 3. start to explore for each episode
for i in range(MAX_EPISODE):
    
    cur_init_ds_ep = env.reset(3)
        
    res_init_buf.append(cur_init_ds_ep)
    
    cur_r_ep = np.zeros(len(user_list))
    cur_p_ep = np.zeros(len(user_list))
    cur_ts_ep = np.zeros(len(user_list))
    cur_ps_ep = np.zeros(len(user_list))
    cur_rs_ep = np.zeros(len(user_list))
    cur_ds_ep = np.zeros(len(user_list))
    cur_ch_ep = np.zeros(len(user_list))
    cur_of_ep = np.zeros(len(user_list))
    
    res_rec_tmp = []
    for j in range(MAX_EPISODE_LEN):
        
        # first try to transmit from current state
        [cur_r, done, cur_p, temp, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of] = env.step_transmit()
        
        res_rec_tmp.append([cur_r, done, cur_p, temp, cur_ts, cur_ps, cur_rs, cur_ds, cur_ch, cur_of])
        cur_r_ep += cur_r
        cur_p_ep += cur_p
        cur_ts_ep += cur_ts
        cur_ps_ep += cur_ps
        cur_rs_ep += cur_rs
        cur_ds_ep += cur_ds
        cur_ch_ep += cur_ch
        cur_of_ep += cur_of
        
        if done:
            res_rec.append(res_rec_tmp)
            print('%d:r:%.4f,p:%.4f,tr:%.4f,pr:%.4f,rev:%.4f,dbuf:%.4f,ch:%.8f,ibuf:%d' % (i, cur_r_ep/MAX_EPISODE_LEN, cur_p_ep/MAX_EPISODE_LEN, cur_ts_ep/MAX_EPISODE_LEN, cur_ps_ep/MAX_EPISODE_LEN, cur_rs_ep/MAX_EPISODE_LEN, cur_ds_ep/MAX_EPISODE_LEN, cur_ch_ep/MAX_EPISODE_LEN, cur_init_ds_ep[0]))
                
name = 'test_1000_' + str(time.asctime( time.localtime(time.time()) )) + '.npy'
np.save(name, [res_rec, res_init_buf])
sess.close()

In [9]:
sess = tf.Session() 
saver = tf.train.import_meta_graph('model/my_test_model.meta')
sess.run(tf.global_variables_initializer())

saver.restore(sess, tf.train.latest_checkpoint('model/'))

graph = tf.get_default_graph()
variable_names = [v.name for v in tf.trainable_variables()]
values = sess.run(variable_names)
print sess.run('fully_connected_2/weights:0')

INFO:tensorflow:Restoring parameters from model/my_test_model
[[0.1545126  0.15780352]
 [0.15616506 0.15971294]
 [0.06559879 0.09880391]
 [0.06487427 0.10064688]
 [0.15168335 0.16229852]
 [0.06257845 0.09748891]
 [0.06430701 0.09924787]
 [0.06548443 0.10012874]
 [0.15782878 0.15414344]
 [0.15429462 0.17596859]
 [0.05945097 0.0991616 ]
 [0.06383383 0.09632684]
 [0.06522399 0.10438306]
 [0.06095436 0.09916781]
 [0.06635379 0.0955418 ]
 [0.06379106 0.09433711]
 [0.06133086 0.09875894]
 [0.06503102 0.09639146]
 [0.15047684 0.17412661]
 [0.15663381 0.15740666]
 [0.17768952 0.15899988]
 [0.15730077 0.14491995]
 [0.05902164 0.10017106]
 [0.06845198 0.09686822]
 [0.15724926 0.14774598]
 [0.1529071  0.16426612]
 [0.1520101  0.15706535]
 [0.16091335 0.16260217]
 [0.15082897 0.16070077]
 [0.06310375 0.09398151]
 [0.06018374 0.10056869]
 [0.05968894 0.10066012]
 [0.15474424 0.16255529]
 [0.15111688 0.1622602 ]
 [0.0838669  0.18973866]
 [0.06870251 0.24295035]
 [0.06702217 0.10505467]
 [0.1529     

In [3]:
np.save('t_08/1', [1])